In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import re
import os
import zipfile
import shutil
import multiprocessing

file_name=r'C:\Users\s3309\Desktop\Investment\html_files\tifrs-fr1-m1-ci-cr-2330-2022Q2.html'

def html_preprocess(file_name):
    with open(file_name, encoding="UTF-8") as f:
        data=f.read()
    soup = BeautifulSoup(data,'html.parser')

    data_all=[[] for i in range(5)]

    for sht in range(3):
        sheet=soup.find_all('table')[sht]
        cate=sheet.find_all('tr')[0].find_all('th')[0].find(class_='en').text.strip()

        i=1
        while i< len(sheet.find_all('tr')):
            if i==1:
                time_window=[]
                for time in sheet.find_all('tr')[i].find_all('th')[2:]:
                    time_window.append(time.find(class_='en').text)

            elif sheet.find_all('tr')[i].find_all('td')[0].text=='' and re.search(r'\d',sheet.find_all('tr')[i+1].find_all('td')[0].text):
                sub_cate=sheet.find_all('tr')[i].find_all('td')[1].find(class_='en').text.strip()

            elif re.search(r'\d',sheet.find_all('tr')[i].find_all('td')[0].text):

                for j,n in enumerate(sheet.find_all('tr')[i].find_all('td')[2:]):
                    data_all[0].append(cate)
                    data_all[1].append(sub_cate)
                    data_all[2].append(sheet.find_all('tr')[i].find_all('td')[1].find(class_='en').text.strip())
                    data_all[3].append(time_window[j])
                    data_all[4].append(n.text)
            i+=1

    table=pd.DataFrame()
    for i,col in enumerate(['Category','Sub-Category','Subject','Period','$']):
        table[col]=data_all[i]

    return table

def xml_preprocess(file_name):
    xml_df=pd.read_xml(file_name)
    xml_df.dropna(subset=['unitRef'],inplace=True)
    ui=xml_df.columns.to_list().index('unitRef')
    target_col=xml_df.columns[ui+1:]
    xml_df['Subject']=pd.NA
    xml_df['$']=pd.NA

    for col in target_col:
        xml_df.loc[~xml_df[col].isna(),'Subject']=col
        xml_df['$'].fillna(xml_df[col],inplace=True)
        xml_df.drop(col,axis=1,inplace=True)

    xml_df=xml_df[['contextRef','Subject','$']]
    xml_df['contextRef']=xml_df['contextRef'].replace(r'From','',regex=True)

    return xml_df


In [3]:
#unzip file
import os
import zipfile

for root, dirs, files in os.walk(r"C:\Users\s3309\Desktop\Investment\todo"):
    for file in files:
        if file.endswith('zip'):
            with zipfile.ZipFile(os.path.join(root,file), 'r') as zip_ref:
                zip_ref.extractall(r'C:\Users\s3309\Desktop\Investment\html_files')
                zip_ref.close()

: 

: 

In [29]:
xml_df=pd.DataFrame()
html_df=pd.DataFrame()

for root, dirs, files in os.walk(r"C:\Users\s3309\Desktop\Investment\todo"):
    for f in files[:30]:
        filename=f.split('.')[0]
        filename=filename.split('-')
        comp=filename[-2]
        qtr=filename[-1]
        abspath=os.path.join(root,f)

        if f.endswith('.xml'):
            temp=xml_preprocess(abspath)
            temp['Comp']=comp
            temp['Qtr']=qtr
            xml_df=pd.concat([xml_df,temp])
        else:
            temp=html_preprocess(abspath)
            temp['Comp']=comp
            temp['Qtr']=qtr
            html_df=pd.concat([html_df,temp])         
            


In [4]:
import sqlite3
import pandas as pd

conn=sqlite3.connect('test')
test=pd.read_sql('SELECT * FROM NEW_XML_DF', conn)
html_test=pd.read_sql('SELECT * FROM NEW_HTML_DF', conn)

DatabaseError: Execution failed on sql 'SELECT * FROM NEW_XML_DF': no such table: NEW_XML_DF

In [7]:
html_test=pd.read_sql('SELECT * FROM NEW_HTML_DF', conn)

In [4]:
import re
html_df=html_test.copy()

def html_process(html_df):
    html_df['$']=html_df['$'].replace(r',|\)|\n','',regex=True)
    html_df['$']=html_df['$'].replace(r'^\(','-',regex=True)
    html_df['$']=html_df['$'].replace(r'',0,regex=True)
    html_df['$']=html_df['$'].astype(float)
    html_df['$']=html_df['$'].apply(lambda x:x*1000)
    html_df['Mapping Col']=html_df['Subject'].apply(lambda x:re.sub(' |-','',x).lower())
    html_df['Mapping Col']=html_df['Mapping Col'].replace(r'settlement$','settlements',regex=True)
    html_df['Mapping Col']=html_df['Mapping Col'].replace(r',net','net',regex=True)
    subcate_d={subject:subcate for subject,subcate in zip(html_df['Mapping Col'],html_df['Sub-Category'])}
    return html_df,subcate_d

html_df,subcate_d=html_process(html_df)


In [5]:
xml_df=test.copy()

xml_df['Mapping Col']=xml_df['Subject'].apply(lambda x:x.lower())
xml_df['Mapping Col']=xml_df['Mapping Col'].apply(lambda x:x.split('-')[0])
xml_df['Sub-Category']=xml_df['Mapping Col'].map(subcate_d)


In [6]:
unisubcate=xml_df[xml_df['Sub-Category'].isna()]['Mapping Col'].unique()
html_unique=html_test['Mapping Col'].unique()

html_unique.sort()
unisubcate.sort()

In [3]:
files = open("problem.txt")
file_list =  files.readlines()

In [34]:
print(html_unique[:10])
print(unisubcate[:10])

['(impairmentlossonassets)reversalofimpairmentlossonassets'
 '(non-current)assets(ordisposalgroups)classifiedasheld\nforsale,net'
 '(non-current)assets(ordisposalgroups)classifiedasheldfordistributiontoowners,net'
 '(non-current)assets(ordisposalgroups)classifiedasheldforsale,net'
 '(reversalof)impairmentlossonnon-financialassets,net' 'acceptances'
 'acceptancesreceivable' 'accounts\npayable'
 'accounts\npayabletorelatedparties' 'accounts\nreceivable,net']
['accountspayable-accountspayable'
 'accountspayableforsettlementfromnonbrokeredtrading'
 'accountspayableforsettlements' 'accountspayableothers'
 'accountspayabletorelatedparties-accountspayabletorelatedparties'
 'accountsreceivableduefromrelatedpartiesnet'
 'accountsreceivableforsettlementfromnonbrokeredtrading'
 'accountsreceivableforsettlements' 'accountsreceivablenet'
 'accountsreceivableothers']


In [35]:
unisubcate[2] is 

'accountspayableforsettlements'

In [39]:
unisubcate=xml_df[xml_df['Sub-Category'].isna()]['Mapping Col'].unique()

In [21]:
import pickle

with open('subcate_dict.pkl', 'wb') as f:
    pickle.dump(subcate_d, f)

In [33]:
with open(r'problem.txt', 'w') as fp:
    fp.write('\n'.join(unisubcate))

In [34]:
import sqlite3

conn=sqlite3.connect('test')
xml_df.to_sql('XML_DF',conn)

26851

In [6]:
import test
from itertools import repeat
import pandas as pd
import multiprocessing

xml_df=pd.DataFrame()
html_df=pd.DataFrame()
for root, dirs, files in os.walk(r"C:\Users\s3309\Desktop\Investment\html_files"):
    file_list=files

with multiprocessing.Pool(8) as p:
    result=p.map(test.file_process,file_list)

'''
for i,n in enumerate(result):

    if n is None:
        continue
    if 'Sub-Category' in n.columns:
        html_df=pd.concat([html_df,n])
    else:
        xml_df=pd.concat([xml_df,n])

import sqlite3

conn=sqlite3.connect('test')
html_df.to_sql('HTML_DF',conn,if_exists='append')
xml_df.to_sql('XML_DF',conn,if_exists='append')

'''

"\nfor i,n in enumerate(result):\n\n    if n is None:\n        continue\n    if 'Sub-Category' in n.columns:\n        html_df=pd.concat([html_df,n])\n    else:\n        xml_df=pd.concat([xml_df,n])\n\nimport sqlite3\n\nconn=sqlite3.connect('test')\nhtml_df.to_sql('HTML_DF',conn,if_exists='append')\nxml_df.to_sql('XML_DF',conn,if_exists='append')\n\n"

In [1]:
import sqlite3
conn=sqlite3.connect('test')


In [1]:
import test
from itertools import repeat
import pandas as pd
import multiprocessing

xml_df=pd.DataFrame()
html_df=pd.DataFrame()
for root, dirs, files in os.walk(r"C:\Users\s3309\Desktop\Investment\html_files"):
    file_list=files

with multiprocessing.Pool(8) as p:
    result=p.map(test.file_process,file_list)

print('Result finish----')


for i,n in enumerate(result):
    print(i)
    try:
        if n is None:
            continue
        if 'Sub-Category' in n.columns:
            html_df=pd.concat([html_df,n])
        else:
            xml_df=pd.concat([xml_df,n])
    except:
        continue
'''
import sqlite3

conn=sqlite3.connect('test')
try:
    html_df.to_sql('NEW_HTML_DF',conn,if_exists='append')
    print('html finish')
    xml_df.to_sql('NEW_XML_DF',conn,if_exists='append')
except:
    print('Error')

'''

: 

: 

In [3]:
%pip install --upgrade ipython

  Using cached pure_eval-0.2.2-py3-none-any.whl (11 kB)
  Attempting uninstall: ipython
    Found existing installation: ipython 7.16.1
    Uninstalling ipython-7.16.1:
      Successfully uninstalled ipython-7.16.1
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
finmind 1.5.3 requires ipython~=7.16.1, but you have ipython 8.6.0 which is incompatible.
finmind 1.5.3 requires numpy~=1.19.5, but you have numpy 1.22.3 which is incompatible.
finmind 1.5.3 requires pandas~=1.1.5, but you have pandas 1.5.1 which is incompatible.
finmind 1.5.3 requires requests~=2.23.0, but you have requests 2.26.0 which is incompatible.
finmind 1.5.3 requires setuptools~=49.2.1, but you have setuptools 60.2.0 which is incompatible.
You should consider upgrading via the 'C:\Users\s3309\AppData\Local\Programs\Python\Python39\dlenv\Scripts\python.exe -m pip install --upgrade pip' command.


In [3]:
import sqlite3
import pandas as pd

conn=sqlite3.connect('test')
html_test=pd.read_sql('SELECT * FROM CN_HTML_DF', conn)
conn.close()

In [5]:
import shutil
for root,dir,files in os.walk(r'C:\Users\s3309\Desktop\Investment\completed'):
    for f in files:
        shutil.move(os.path.join(root,f),os.path.join(r'C:\Users\s3309\Desktop\Investment\html_files',f))

In [6]:
help(multiprocessing.Pool)

Help on method Pool in module multiprocessing.context:

Pool(processes=None, initializer=None, initargs=(), maxtasksperchild=None) method of multiprocessing.context.DefaultContext instance
    Returns a process pool object



In [41]:
xml_comp=[x for x in xml_df['Comp'].unique() if not x in xml_df2['Comp'].unique()]
html_comp=[x for x in html_df['Comp'].unique() if not x in html_df2['Comp'].unique()]






In [1]:
from multiprocessing import Pool

def f(x):
    return x*x

if __name__ == '__main__':
    with Pool(5) as p:
        print(p.map(f, [1, 2, 3]))

: 

: 

In [2]:
xml_df


,contextRef,Subject,$,Comp,Qtr
120,AsOf20130331,CashOnHand,9828116000.0,2801,2013Q1
121,AsOf20121231,CashOnHand,9719048000.0,2801,2013Q1
122,AsOf20120331,CashOnHand,9269632000.0,2801,2013Q1
123,AsOf20120101,CashOnHand,8820740000.0,2801,2013Q1
124,AsOf20130331,ChecksForClearing,19125995000.0,2801,2013Q1
...,...,...,...,...,...
564,20140101To20141231,ShareOfProfitLossForTheQuarterOfAssociatesAndJ...,0.0,5874,2014Q4
565,AsOf20141231,EndingBalanceOfAssetsOfSubsidiariesWhoseFinanc...,0.0,5874,2014Q4
566,20140101To20141231,YearToDateComprehensiveIncomeOfSubsidiariesWho...,0.0,5874,2014Q4
567,AsOf20141231,EndingBalanceOfInvestmentsAccountedForUsingEqu...,0.0,5874,2014Q4


In [6]:
import multiprocessing  
import random
from multiprocessing import Pool
import defs

if __name__ == '__main__':
    pool = Pool()
    to_factor = [ random.randint(100000, 50000000) for i in range(20)]
    results = pool.map(defs.prime_factor, to_factor)
    for value, factors in zip(to_factor, results):
        print("The factors of {} are {}".format(value, factors))

The factors of 14326878 are [7163439, 2387813, 67, 35639, 157, 227]
The factors of 45512257 are [45512257, 7, 6501751, 37, 175723]
The factors of 34349663 are [34349663, 19, 1807877, 499, 3623]
The factors of 28136668 are [7034167, 7, 1004881, 83, 12107]
The factors of 26872128 are [419877, 139959, 46653, 15551]
The factors of 49737876 are [12434469, 4144823, 103, 40241]
The factors of 15761876 are [3940469, 13, 303113, 41, 7393]
The factors of 4998695 are [4998695, 5, 999739, 13, 76903, 53, 1451]
The factors of 14457694 are [7228847, 89, 81223]
The factors of 41446658 are [20723329, 11, 1883939]
The factors of 31564669 are [31564669]
The factors of 5493766 are [2746883, 43, 63881, 127, 503]
The factors of 24691622 are [12345811]
The factors of 30767224 are [3845903]
The factors of 2827227 are [2827227, 942409, 13, 72493]
The factors of 13199849 are [13199849, 13, 1015373, 193, 5261]
The factors of 44158000 are [2759875, 5, 551975, 5, 110395, 5, 22079]
The factors of 46646171 are [4664

In [4]:
driver_func()

Here we start


AttributeError: module 'test' has no attribute 'double'

In [2]:
nums=[-1,0,1,2,-1,-4]
nums.sort()
nums

[-4, -1, -1, 0, 1, 2]

In [4]:
nums=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
nums.sort()

i=0
res=[]
while i<=len(nums)-2:
    print(i)
    j=i+1
    k=len(nums)-1
    ans=[]
    
    while j<k:
        print(j,k)
        if nums[i]+nums[j]+nums[k]<0:
            j+=1
        
        elif nums[i]+nums[j]+nums[k]>0:
            k-=1
        
        else:
            ans=[nums[i],nums[j],nums[k]]
            if not ans in res:
                res.append(ans)
                
            j+=1
            k-=1
    i+=1

    res

0
1 2999
2 2998
3 2997
4 2996
5 2995
6 2994
7 2993
8 2992
9 2991
10 2990
11 2989
12 2988
13 2987
14 2986
15 2985
16 2984
17 2983
18 2982
19 2981
20 2980
21 2979
22 2978
23 2977
24 2976
25 2975
26 2974
27 2973
28 2972
29 2971
30 2970
31 2969
32 2968
33 2967
34 2966
35 2965
36 2964
37 2963
38 2962
39 2961
40 2960
41 2959
42 2958
43 2957
44 2956
45 2955
46 2954
47 2953
48 2952
49 2951
50 2950
51 2949
52 2948
53 2947
54 2946
55 2945
56 2944
57 2943
58 2942
59 2941
60 2940
61 2939
62 2938
63 2937
64 2936
65 2935
66 2934
67 2933
68 2932
69 2931
70 2930
71 2929
72 2928
73 2927
74 2926
75 2925
76 2924
77 2923
78 2922
79 2921
80 2920
81 2919
82 2918
83 2917
84 2916
85 2915
86 2914
87 2913
88 2912
89 2911
90 2910
91 2909
92 2908
93 2907
94 2906
95 2905
96 2904
97 2903
98 2902
99 2901
100 2900
101 2899
102 2898
103 2897
104 2896
105 2895
106 2894
107 2893
108 2892
109 2891
110 2890
111 2889
112 2888
113 2887
114 2886
115 2885
116 2884
117 2883
118 2882
119 2881
120 2880
121 2879
122 2878
123 2877

: 

: 